<a href="https://colab.research.google.com/github/Ratpol7/Minecraft_Server/blob/main/test03_Minecraft_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import re
import json

# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 17
# !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
# !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk17 gets installed."
!sudo apt-get install openjdk-17-jre-headless &>/dev/null && echo "Yay! Openjdk17 has been successfully installed." || echo "Failed to install OpenJdk17."
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "17" :
  print("Openjdk17 is working correctly, you are good to go.")
else:
  print("Openjdk17 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance. Minecraft 1.17 and above might fail to launch.")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Change directory to the Minecraft server folder on Google Drive
%cd "/content/drive/My Drive/Minecraft-server"
!ls #list the directory contents (to verify that working directory was changed)

# Import config file.
if os.path.isfile("colabconfig.json"):
  colabconfig = json.load(open("colabconfig.json"))
else:
  colabconfig = {"server_type": "generic"} # using default, if config doesn't exists.
  json.dump(colabconfig, open("colabconfig.json",'w'))

# Server jar names.
jar_list = {'paper': 'server.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar'}
jar_name = jar_list[colabconfig["server_type"]]

# Java arguments.
if colabconfig["server_type"] == "paper":
  server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"
else:
  server_flags = "" # aiker's flags might negatively impact performance on non-paper servers.
memory_allocation = "-Xmx6144M -Xms6144M"

# Chose the tunnle service you want to use
# Available options: ngrok, argo
tunnel_service = "ngrok"#"argo"
print("Procedding to use", tunnel_service)

if tunnel_service == "ngrok":
  !pip -q install pyngrok
  from pyngrok import conf, ngrok
  
  # Ask for the ngrok authtoken
  print("Get your authtoken from https://dashboard.ngrok.com/auth")
  import getpass
  authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
  ! ngrok authtoken $authtoken # login to ngrok

  # Sets default ngrok region
  conf.get_default().region = 'ap'  # Change this to whichever region you want

  # Connect to ngrok
  url = ngrok.connect(23428, 'tcp')
  print('Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

  print('Starting server...')
  !java $memory_allocation $server_flags -jar $jar_name nogui

else:
  # Download & make argo executable
  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
  print('Starting server...')
  !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:23428 | java $memory_allocation $server_flags -jar $jar_name nogui


apt cache successfully updated
Yay! Openjdk17 has been successfully installed.
Openjdk17 is working correctly, you are good to go.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server
banned-ips.json		   commands.yml     server.properties
banned-players.json	   eula.txt	    spigot.yml
bukkit.yml		   help.yml	    usercache.json
cache			   logs		    version_history.json
cloudflared-linux-amd64    ops.json	    whitelist.json
cloudflared-linux-amd64.1  paper.yml	    world
cloudflared-linux-amd64.2  permissions.yml  world_nether
cloudflared-linux-amd64.3  plugins	    world_the_end
colabconfig.json	   server.jar
Procedding to use ngrok
Get your authtoken from https://dashboard.ngrok.com/auth
··········
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Your server address is 0.tcp.ap.ngrok.io:12578
Starting server...
System Info: Java 17 (OpenJDK 64-Bit Server VM 1

t=2022-04-22T09:43:46+0000 lvl=warn msg="failed to open private leg" id=238aea13c445 typ=proxy privaddr=localhost:23428 err="dial tcp 127.0.0.1:23428: connect: connection refused"


[09:43:46 INFO]: Preparing spawn area: 0%
[09:43:46 INFO]: Preparing spawn area: 0%
[09:43:46 INFO]: Preparing spawn area: 0%
[09:43:46 INFO]: Preparing spawn area: 0%
[09:43:47 INFO]: Preparing spawn area: 0%
[09:43:47 INFO]: Time elapsed: 2157 ms
[09:43:47 INFO]: Preparing start region for dimension minecraft:the_nether
[09:43:47 INFO]: Preparing spawn area: 0%
[09:43:47 INFO]: Preparing spawn area: 0%
[09:43:48 INFO]: Preparing spawn area: 0%
[09:43:48 INFO]: Time elapsed: 1297 ms
[09:43:48 INFO]: Preparing start region for dimension minecraft:the_end
[09:43:48 INFO]: Preparing spawn area: 0%
[09:43:49 INFO]: Preparing spawn area: 0%
[09:43:49 INFO]: Time elapsed: 592 ms
[09:43:49 INFO]: Running delayed init tasks
[09:43:49 INFO]: Done (6.737s)! For help, type "help"
[09:43:49 INFO]: Timings Reset
>....

t=2022-04-22T09:43:54+0000 lvl=warn msg="failed to open private leg" id=166e6d2b7d95 typ=proxy privaddr=localhost:23428 err="dial tcp 127.0.0.1:23428: connect: connection refused"


[09:48:15 INFO]: Unknown command. Type "/help" for help.
[09:48:34 INFO]: Unknown command. Type "/help" for help.
[09:48:48 INFO]: Unknown command. Type "/help" for help.
[09:48:52 INFO]: --------- Help: Index ---------------------------
[09:48:52 INFO]: Use /help [n] to get page n of help.
[09:48:52 INFO]: Aliases: Lists command aliases
[09:48:52 INFO]: Bukkit: All commands for Bukkit
[09:48:52 INFO]: Minecraft: All commands for Minecraft
[09:48:52 INFO]: /advancement: A Mojang provided command.
[09:48:52 INFO]: /attribute: A Mojang provided command.
[09:48:52 INFO]: /ban: A Mojang provided command.
[09:48:52 INFO]: /ban-ip: A Mojang provided command.
[09:48:52 INFO]: /banlist: A Mojang provided command.
[09:48:52 INFO]: /bossbar: A Mojang provided command.
[09:48:52 INFO]: /clear: A Mojang provided command.
[09:48:52 INFO]: /clone: A Mojang provided command.
[09:48:52 INFO]: /data: A Mojang provided command.
[09:48:52 INFO]: /datapack: A Mojang provided command.
[09:48:52 INFO]: /deb

In [1]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


In [3]:
# Replace "1.17" with your desired server version.
# Available versions:
# - 1.18 <!> Paper still in experimental
# - 1.17.1
# - 1.17
# - 1.16.4
# - 1.16.3
# - 1.15.2
# - 1.14.4
# - 1.13.2
# - 1.12.2
# - 1.11.2
# - 1.10.2
# - 1.9.4
# - 1.8.8
# Newer versions might work too, however this isn't guaranteed.
version = '1.17.1'

#Chose server type. Currently available versions: fabric, paper
server_type = 'paper'

# Print experimental build warning
if version == '1.18' and server_type == 'paper':
  print("<!> Paper 1.18 builds are still experimental. Make regular Backups!")

from google.colab import drive
import requests
import json


drive.mount('/content/drive')

# Create the directory which will be used for the server
! mkdir "/content/drive/My Drive/Minecraft-server"
%cd "/content/drive/My Drive/Minecraft-server"

# Internal init...


a = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version)
#print(a.json()["builds"][-1])
b = requests.get("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]))
#print(b.json()["downloads"]["application"]["name"])
print("https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"])

paperURL = "https://papermc.io/api/v2/projects/paper/versions/" + version + "/builds/" + str(a.json()["builds"][-1]) + "/downloads/" + b.json()["downloads"]["application"]["name"]

jar_name = {'paper': 'server.jar', 'fabric': 'fabric-installer.jar'}
url = {
    'paper': (paperURL),
    'fabric': 'https://maven.fabricmc.net/net/fabricmc/fabric-installer/0.7.4/fabric-installer-0.7.4.jar'
    }

print('Downloading to Google Drive...')

r = requests.get(url[server_type])

if r.status_code is 200:
  with open('/content/drive/My Drive/Minecraft-server/' + jar_name[server_type], 'wb') as f:
    f.write(r.content)
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')

# Running specific install path.
if server_type == 'fabric':
  !java -jar fabric-installer.jar server -mcversion $version -downloadMinecraft

# Saving config
colabconfig = {"server_type": server_type}
json.dump(colabconfig, open("colabconfig.json",'w'))

print('Done!') # todo: Would show even after erroring.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/My Drive/Minecraft-server’: File exists
/content/drive/My Drive/Minecraft-server
https://papermc.io/api/v2/projects/paper/versions/1.17.1/builds/411/downloads/paper-1.17.1-411.jar
Done!


**Automatically accept the EULA**

In [4]:
# Please read the file stored in your server folder before running this command. 
# Also, go to https://www.minecraft.net/en-us/eula to read Minecraft's EULA.

# Make sure Drive is mounted
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/My Drive/Minecraft-server"
!echo "eula=true" >> eula.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Minecraft-server


# Debug
SSH access to host OS - Thanks to [colab-ssh](https://github.com/WassimBenzarti/colab-ssh).

In [7]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "0.tcp.ap.ngrok.io:12578" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

In [8]:
#Get public address (ngrok)
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [9]:
## For inspecting the minecraft server directory ##
%cd "/content/drive/My Drive/Minecraft-server"
!ls


/content/drive/My Drive/Minecraft-server
banned-ips.json		   colabconfig.json  server.jar
banned-players.json	   commands.yml      server.properties
bukkit.yml		   eula.txt	     spigot.yml
cache			   help.yml	     usercache.json
cloudflared		   logs		     version_history.json
cloudflared-linux-amd64.1  ops.json	     whitelist.json
cloudflared-linux-amd64.2  paper.yml	     world
cloudflared-linux-amd64.3  permissions.yml   world_nether
cloudflared.log		   plugins	     world_the_end


In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "<PUT_YOUR_PASSWORD_HERE>" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="<PUT_YOUR_REPOSITORY_URL_HERE>" #@param {type: "string"}
init_git_cloudflared(repository_url)